In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from scipy.stats import zscore
from itertools import combinations



# Load the 'boldp1.csv' file with no header
boldp1_path = 'boldp8.csv'  # Replace with your file path
boldp1_df = pd.read_csv(boldp1_path, header=None)

# Load the 'Stimuli.csv' file
stimuli_path = 'Stimuli.csv'  # Replace with your file path
stimuli_df = pd.read_csv(stimuli_path)

# Filter the data for participant 1
stimuli_participant1_df = stimuli_df[stimuli_df['participant'] == 8]

# Append the filtered Stimuli data to the boldp1 dataset
combined_df = pd.concat([boldp1_df, stimuli_participant1_df.reset_index(drop=True)], axis=1)

# Grouping the data by 'word' and 'epoch' (scanning session)
voxel_columns = boldp1_df.columns
grouped_data = combined_df.groupby(['word', 'epoch'])[voxel_columns].mean()

# Preparing to calculate correlations across all unique pairs of epochs
epoch_pairs = list(combinations(grouped_data.index.levels[1].unique(), 2))

# Initializing a dictionary to store the correlation coefficients for each voxel
voxel_correlations = {voxel: [] for voxel in voxel_columns}

# Calculating the Pearson's correlation for each voxel across all epoch pairs
for word in grouped_data.index.levels[0]:
    word_data = grouped_data.loc[word]
    for epoch1, epoch2 in epoch_pairs:
        if epoch1 in word_data.index and epoch2 in word_data.index:
            epoch1_data = word_data.loc[epoch1]
            epoch2_data = word_data.loc[epoch2]
            for voxel in voxel_columns:
                corr = np.corrcoef(epoch1_data[voxel], epoch2_data[voxel])[0, 1]
                voxel_correlations[voxel].append(corr)

# Calculating the mean of these correlation coefficients for each voxel
mean_correlations = {voxel: np.nanmean(corrs) for voxel, corrs in voxel_correlations.items() if corrs}

# Sorting the voxels by their stability measure (mean correlation) and selecting the top 500
top_500_stable_voxels = pd.Series(mean_correlations).sort_values(ascending=False).head(500)


In [2]:
import pandas as pd
import numpy as np

# Load the 'boldp1.csv' file with no header
boldp1_path = 'boldp9.csv'  # Replace with your file path
boldp1_df = pd.read_csv(boldp1_path, header=None)

# Load the 'Stimuli.csv' file
stimuli_path = 'Stimuli.csv'  # Replace with your file path
stimuli_df = pd.read_csv(stimuli_path)

# Filter the data for participant 1
stimuli_participant1_df = stimuli_df[stimuli_df['participant'] == 9]

# Append the filtered Stimuli data to the boldp1 dataset
combined_df = pd.concat([boldp1_df, stimuli_participant1_df.reset_index(drop=True)], axis=1)

# Pivot the data to create a matrix for each voxel
voxel_columns = boldp1_df.columns
reshaped_data = combined_df.pivot_table(index='epoch', columns='word', values=voxel_columns)

# Calculating the stability score for each voxel
stability_scores_full = {}
for voxel in voxel_columns:
    voxel_matrix = reshaped_data[voxel]
    pairwise_correlations = []
    for row1 in range(voxel_matrix.shape[0]):
        for row2 in range(row1 + 1, voxel_matrix.shape[0]):
            corr = voxel_matrix.iloc[row1].corr(voxel_matrix.iloc[row2])
            pairwise_correlations.append(corr)
    stability_scores_full[voxel] = np.nanmean(pairwise_correlations)

# Sorting the voxels by their stability score
sorted_stability_scores_full = pd.Series(stability_scores_full).sort_values(ascending=False)

# Display the top and bottom stability scores (optional)
print(sorted_stability_scores_full.head())
print(sorted_stability_scores_full.tail())


6724    0.210825
5402    0.207444
6743    0.205482
6744    0.198242
6723    0.186941
dtype: float64
9373   -0.095583
1714   -0.096747
4700   -0.097005
424    -0.098527
440    -0.109958
dtype: float64


In [3]:
# Extracting the top 500 most stable voxels
top_500_stable_voxels = sorted_stability_scores_full.head(500)

# Get the indices (column numbers) of the top 500 stable voxels
top_500_voxel_indices = top_500_stable_voxels.index.tolist()

# Select only the columns corresponding to the top 500 stable voxels
top_500_voxels_data = boldp1_df[top_500_voxel_indices]

In [4]:
# Filter the data for participant 1
stimuli_df = stimuli_df[stimuli_df['participant'] == 9]

# Extracting the 'word' column
words_p1 = stimuli_participant1_df['word']

# Append the 'word' column to the top 500 voxels data
top_500_voxels_data_with_words = pd.concat([top_500_voxels_data, words_p1.reset_index(drop=True)], axis=1)

top_500_voxels_data_with_words.head

<bound method NDFrame.head of         6724     5402     6743     6744      6723     5384     5403      3410  \
0    3.71430  2.42940  3.19070  1.90230  3.649100  2.13550  2.14330 -0.157480   
1    3.60840  2.44150  3.36600  2.11160  4.443100  2.89610  1.67130  0.429330   
2    2.68890  2.18450  3.17330  1.83650  4.000900  1.12390  1.56530  0.921320   
3    0.78590  1.36460  1.37810  0.49488  1.934200  1.21410  0.52920 -0.014492   
4    3.29330  2.05270  3.24890  2.27120  3.631500  2.31910  1.45230  0.891690   
..       ...      ...      ...      ...       ...      ...      ...       ...   
355  0.58243  0.23522  0.97779  0.40188  2.017400  1.43320 -0.21887  0.025042   
356  1.51910  0.14681  1.74610  1.83090  1.469500  0.48209  0.58705 -0.123620   
357  3.54430  1.53550  2.26840  1.96160  3.275000  0.58257  1.24130  0.556460   
358 -0.11083 -0.17226 -0.86546 -0.96015 -0.093343  0.69813 -0.53656 -0.284280   
359  3.12820  2.20940  2.47560  1.68970  3.464100  2.60690  2.22890  1.018100  

In [5]:
# Grouping by 'word' and calculating the mean for each voxel
grouped_data = top_500_voxels_data_with_words.groupby('word').mean()

# Resetting the index to include the 'word' column in the dataframe
grouped_data.reset_index(inplace=True)

# Now, grouped_data contains the averaged voxel data for each word, resulting in a 60 x 501 dataset
grouped_data.head

<bound method NDFrame.head of             word      6724      5402      6743      6744      6723      5384  \
0       airplane  2.612983  1.938133  2.143167  1.782988  2.886317  1.187397   
1            ant  2.823983  1.704215  2.398550  1.769492  3.242983  1.356565   
2      apartment  3.254400  1.639865  2.685117  2.323567  3.647800  1.947260   
3           arch  3.115017  1.575548  2.267567  1.896088  3.370783  1.316127   
4            arm  1.125443  0.329148  0.611893  0.536145  1.214922  1.001963   
5           barn  3.118467  1.704128  2.662933  1.699667  3.492150  1.999217   
6           bear  2.348917  0.944358  1.482772  1.377768  2.266767  1.098213   
7            bed  2.163087  0.788113  0.863358  1.214645  1.876604  1.739213   
8            bee  1.585345  0.727118  1.094387  0.902499  1.608395  0.449289   
9         beetle  2.466380  0.924330  1.625493  1.223819  2.168117  0.919691   
10          bell  1.562060  1.358169  1.276687  1.108511  1.833275  1.115695   
11       b

In [6]:
# Dropping the 'word' column to leave only the voxel data
voxel_data_only = grouped_data.drop('word', axis=1)
transposed_data = voxel_data_only.T
# Calculating the correlation matrix
correlation_matrix = transposed_data.corr()
correlation_matrix

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,1.000000,0.728910,0.619199,0.645485,0.499099,0.629685,0.451695,0.596356,0.631329,0.563862,...,0.651426,0.539979,0.512286,0.337667,0.521269,0.093322,0.656985,0.641108,0.460071,0.519815
1,0.728910,1.000000,0.605222,0.610469,0.613272,0.700879,0.512332,0.544930,0.654079,0.606011,...,0.766074,0.581216,0.640934,0.403408,0.538800,0.114154,0.599943,0.672708,0.513919,0.558631
2,0.619199,0.605222,1.000000,0.795689,0.372581,0.792710,0.523802,0.690165,0.466241,0.516281,...,0.610346,0.336472,0.458625,0.536448,0.580274,-0.011874,0.771128,0.461891,0.361368,0.622676
3,0.645485,0.610469,0.795689,1.000000,0.517161,0.719381,0.425856,0.601821,0.471326,0.499377,...,0.636861,0.390217,0.417902,0.563133,0.560896,-0.029694,0.718866,0.516418,0.461813,0.692363
4,0.499099,0.613272,0.372581,0.517161,1.000000,0.439888,0.278727,0.335666,0.349628,0.347909,...,0.645294,0.269900,0.384484,0.425626,0.325265,-0.106577,0.374423,0.487274,0.284430,0.426447
5,0.629685,0.700879,0.792710,0.719381,0.439888,1.000000,0.582941,0.723070,0.566033,0.600352,...,0.708883,0.465539,0.510621,0.514060,0.653962,0.043648,0.727984,0.500559,0.470289,0.571043
6,0.451695,0.512332,0.523802,0.425856,0.278727,0.582941,1.000000,0.515398,0.518446,0.570651,...,0.506309,0.455845,0.313971,0.406462,0.689438,0.104656,0.548223,0.269508,0.492885,0.355812
7,0.596356,0.544930,0.690165,0.601821,0.335666,0.723070,0.515398,1.000000,0.513022,0.519838,...,0.586929,0.380551,0.519642,0.422523,0.511047,0.033318,0.610919,0.399731,0.409868,0.410634
8,0.631329,0.654079,0.466241,0.471326,0.349628,0.566033,0.518446,0.513022,1.000000,0.626511,...,0.610717,0.606598,0.501568,0.280796,0.564880,0.192507,0.514390,0.485674,0.464897,0.395462
9,0.563862,0.606011,0.516281,0.499377,0.347909,0.600352,0.570651,0.519838,0.626511,1.000000,...,0.584958,0.675124,0.485450,0.429274,0.728449,0.416773,0.630311,0.342834,0.659675,0.334141


In [7]:
# Ensure the 'word' column from grouped_data is set as the index
grouped_data.set_index('word', inplace=True)

# Transposing the dataframe so that words become columns and voxels become rows
transposed_data = grouped_data.T

# Calculating the correlation matrix between words
word_correlation_matrix = transposed_data.corr()

# Setting the words as the index and column labels of the correlation matrix
word_correlation_matrix.index = grouped_data.index
word_correlation_matrix.columns = grouped_data.index
word_correlation_matrix

word,airplane,ant,apartment,arch,arm,barn,bear,bed,bee,beetle,...,shirt,skirt,spoon,table,telephone,tomato,train,truck,watch,window
word,,,,,,,,,,,,,,,,,,,,,
airplane,1.000000,0.728910,0.619199,0.645485,0.499099,0.629685,0.451695,0.596356,0.631329,0.563862,...,0.651426,0.539979,0.512286,0.337667,0.521269,0.093322,0.656985,0.641108,0.460071,0.519815
ant,0.728910,1.000000,0.605222,0.610469,0.613272,0.700879,0.512332,0.544930,0.654079,0.606011,...,0.766074,0.581216,0.640934,0.403408,0.538800,0.114154,0.599943,0.672708,0.513919,0.558631
apartment,0.619199,0.605222,1.000000,0.795689,0.372581,0.792710,0.523802,0.690165,0.466241,0.516281,...,0.610346,0.336472,0.458625,0.536448,0.580274,-0.011874,0.771128,0.461891,0.361368,0.622676
arch,0.645485,0.610469,0.795689,1.000000,0.517161,0.719381,0.425856,0.601821,0.471326,0.499377,...,0.636861,0.390217,0.417902,0.563133,0.560896,-0.029694,0.718866,0.516418,0.461813,0.692363
arm,0.499099,0.613272,0.372581,0.517161,1.000000,0.439888,0.278727,0.335666,0.349628,0.347909,...,0.645294,0.269900,0.384484,0.425626,0.325265,-0.106577,0.374423,0.487274,0.284430,0.426447
barn,0.629685,0.700879,0.792710,0.719381,0.439888,1.000000,0.582941,0.723070,0.566033,0.600352,...,0.708883,0.465539,0.510621,0.514060,0.653962,0.043648,0.727984,0.500559,0.470289,0.571043
bear,0.451695,0.512332,0.523802,0.425856,0.278727,0.582941,1.000000,0.515398,0.518446,0.570651,...,0.506309,0.455845,0.313971,0.406462,0.689438,0.104656,0.548223,0.269508,0.492885,0.355812
bed,0.596356,0.544930,0.690165,0.601821,0.335666,0.723070,0.515398,1.000000,0.513022,0.519838,...,0.586929,0.380551,0.519642,0.422523,0.511047,0.033318,0.610919,0.399731,0.409868,0.410634
bee,0.631329,0.654079,0.466241,0.471326,0.349628,0.566033,0.518446,0.513022,1.000000,0.626511,...,0.610717,0.606598,0.501568,0.280796,0.564880,0.192507,0.514390,0.485674,0.464897,0.395462


In [8]:
df = pd.DataFrame(word_correlation_matrix)
df.to_csv('p9_bold_sim.csv')